In [1]:
%matplotlib inline
import os
import ftplib
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.feature as cfeat
import cartopy.crs as ccrs

In [2]:
# load paths
website = 'ftp-cdc.dwd.de'
webdir = '/pub/CDC/observations_germany/climate/daily/kl/recent/'
zipdir = 'F:\\Documenten\\Documenten\\CDC\\zip\\'
txtdir = 'F:\\Documenten\\Documenten\\CDC\\txt\\'

In [3]:
## download zipfiles
ftp = ftplib.FTP(website)
ftp.login()
ftp.cwd(webdir)

dirlist = []
filenames = ftp.nlst()

for filename in filenames:
    local_filename = os.path.join(zipdir, filename)
    file = open(local_filename, 'wb')
    ftp.retrbinary('RETR '+ filename, file.write)
    file.close()
    
ftp.quit()
print 'done downloading zips'

done downloading zips


In [4]:
# unpack zipfiles
local_filenames = os.listdir(zipdir)
for filename in local_filenames:
    if filename.endswith('.zip'):
        fullname = os.path.join(zipdir, filename)
        fh = open(fullname, 'rb')
        z = zipfile.ZipFile(fh)
        name = z.namelist()[6]
        z.extract(name, txtdir)
        fh.close()
print 'done unpacking zips'

done unpacking zips


In [4]:
# read txt files into pandas panel
txtnames = [txtdir + txtname for txtname in os.listdir(txtdir)]

dflist = [pd.read_csv(txtname, delimiter = ';', engine = 'python', index_col = ' MESS_DATUM') for txtname in txtnames]
df_concat = pd.concat(dflist, axis = 0)
df_concat.index = [df_concat.index,df_concat.STATIONS_ID]

data = df_concat.to_panel()
data = data.transpose(2,1,0)

# convert index to datetime
data.major_axis = pd.to_datetime(data.major_axis, format = '%Y%m%d')

# read metadata
metadata = pd.read_fwf(zipdir + 'KL_Tageswerte_Beschreibung_Stationen.txt', 
                       index_col = 'Stations_id', 
                       widths = [11, 10, 10, 15, 10, 10, 42, 92],
                       skiprows = [1],
                       encoding = 'utf8')

metadata.columns = ['start','end','height','lat','lon','name','state']

# replace station id with names
#data.items = metadata['name'].loc[data.items.values]

# make subselection on variables
data = data.iloc[:,:,2:-1]

# assign new minor axis names
data.minor_axis = ['T','e','Octo','p','r','wind','Tx','Tn','Tn_10','wind_max','precip','precip_ind','sunhours','snowheight']

# set all -999 to NaN
#data = data[data != -999]

TypeError: unsupported operand type(s) for -: 'str' and 'int'

In [ ]:
stations_Tx = data.loc[:,'2014/12':'2015/02','snowheight']
stations_Tx = stations_Tx[stations_Tx != -999]
stations_Tx = stations_Tx.dropna(axis = 1).mean()
stations_lat = metadata['lat'].loc[data.items.values]
stations_lon = metadata['lon'].loc[data.items.values]

In [ ]:
data2 = pd.concat([stations_lat,stations_lon,stations_Tx], axis = 1)
data2.columns = ['lat','lon','Tx']
data2 = data2.dropna()

In [ ]:
# set up map projection
proj = ccrs.Miller()
fig = plt.figure(figsize = (30,15))
ax = plt.axes(projection = proj)

# set edges of the map
ax.set_extent([5, 16, 46.5, 55.5])

# draw map background
scale = '50m'

oceans = cfeat.NaturalEarthFeature('physical', 'ocean', scale, edgecolor = 'black', facecolor = cfeat.COLORS['water'], linewidths=.5)
land = cfeat.NaturalEarthFeature('physical', 'land', scale, edgecolor = 'black', facecolor = cfeat.COLORS['land'], linewidths=.5)
lakes = cfeat.NaturalEarthFeature('physical', 'lakes', scale, edgecolor = 'black', facecolor = cfeat.COLORS['water'], linewidths=.5)
borders = cfeat.NaturalEarthFeature('cultural', 'admin_0_boundary_lines_land', scale, edgecolor = 'black', facecolor = 'none', linewidths=.5)

ax.add_feature(oceans, zorder = 0)
ax.add_feature(land, zorder = 0)
ax.add_feature(borders, zorder = 0)
ax.add_feature(lakes, zorder = 0)

# plot data

#for lon, lat, text in zip(data2.lon, data2.lat, data2.Tx):
#    ax.text(lon, lat, round(text,1), transform = ccrs.PlateCarree(), ha = 'center', va = 'center')

scat = ax.scatter(data2.lon, data2.lat, s = 100, c = data2.Tx, transform = ccrs.PlateCarree(), cmap = plt.cm.Blues_r)
plt.colorbar(scat)